In [24]:
# from ultralytics import YOLO
# from IPython.display import display, Image
# import cv2
# import numpy as np
# # from object_detection import ObjectDetection
# import math
# from sort import *

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

def dist(P, A, B):
    area = abs ( (A[0] - P[0]) * (B[1] - P[1]) - (A[1] - P[1]) * (B[0] - P[0]) )
    AB = ( (A[0] - B[0]) ** 2 + (A[1] - B[1]) ** 2 ) ** 0.5
    return ( area / AB )

f1_A=(388,821)
f1_B=(661,404) #distance 40

f2_A=(582, 876) #id 194
f2_B=(703, 415) #id 91 distance 30

f3_A=(782, 865) #id 60
f3_B=(743, 417) #id 30  distance 30

f4_A=(999, 866) #id 105
f4_B=(789, 417)  #id 85  distacne 30

f5_A=(1183, 850) #id 4
f5_B=(846, 429) #id 29 distance 30

# Initialize Object Detection
# od = ObjectDetection()
model = YOLO('yolov8n.pt')
cap = cv2.VideoCapture("los_angeles.mp4")

frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))	# 영상의 넓이(가로) 프레임
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))	# 영상의 높이(세로) 프레임
frame_size = (frameWidth, frameHeight)

print(frame_size)
# Initialize count
count = 0
center_points_prev_frame = []

danger_car_pt={}
danger_car_id=[]
tracking_objects = {}
track_id = 0
tracker = Sort(max_age=10, min_hits=3, iou_threshold=0.3)
mask = cv2.imread("mask.png")

while True:
    retval, frame = cap.read()
    imgRegion = cv2.bitwise_and(frame,mask)
    count += 1
    if not(retval):
        break

    # Point current frame
    center_points_cur_frame = []

    # Detect objects on frame
    results = model(imgRegion,stream=True)
    detections = np.empty((0,5))
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cx = int((x1+x2) / 2)
            cy = int((y1+y2) / 2)
            center_points_cur_frame.append((cx, cy, 'Danger!'))
            conf = math.ceil((box.conf[0]*100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]
            #print("FRAME N°", count, " ", x, y, w, h)
            # cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
            if currentClass == "car" or currentClass == "truck" or currentClass == "bus" \
                    or currentClass == "motorbike" and conf > 0.3:
                cv2.rectangle(frame, (x1, y1), (x2,y2), (0, 255, 0), 2) 
                currentArray = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((detections,currentArray))

    # Only at the beginning we compare previous and current frame
    

    resultsTracker=tracker.update(detections)
    
    for result in resultsTracker:
        x1,y1,x2,y2,Id = result
        cx = int((x1+x2) / 2)
        cy = int((y1+y2) / 2)
        Id = int(Id)
        # print(cx,cy,Id)
                
    # for object_id, pt in tracking_objects.items():
        if dist((cx,cy), f1_A, f1_B) < 30:
            tracking_objects[Id]=(cx,cy,'f1')
        
        elif dist((cx,cy), f2_A, f2_B) < 30:
            tracking_objects[Id]=(cx,cy,'f2')
            
        elif dist((cx,cy), f3_A, f3_B) < 30:
            tracking_objects[Id]=(cx,cy,'f3')
        
        elif dist((cx,cy), f4_A, f4_B) < 30:
            tracking_objects[Id]=(cx,cy,'f4')
        
        elif dist((cx,cy), f5_A, f5_B) < 30:
            tracking_objects[Id]=(cx,cy,'f5')
        
        else:
            tracking_objects[Id]=(cx,cy,'Danger!')
            if Id in danger_car_pt:
                danger_car_pt[Id] +=1
                if danger_car_pt[Id] > 30 and id not in danger_car_id:
                    danger_car_id.append(Id)
                    
            else:
                danger_car_pt[Id] = 1
                
            
        
            
            
    for object_id, pt in tracking_objects.items():
        tmp= pt[2]+'['+str(object_id)+']'
        cv2.circle(frame, (pt[0],pt[1]), 5, (0, 0, 255), -1)
        cv2.putText(frame, tmp, (pt[0], pt[1] - 7), 0, 0.8, (0, 0, 255), 2)
        
        

    
                
    
    

    # print("Tracking objects")
    # # print(tracking_objects)


    # print("CUR FRAME LEFT PTS")
    # print(center_points_cur_frame)
    
#     cv2.circle(frame, (1,1), 5, (0, 0, 255), -1)
    cv2.imshow("Frame", frame)
    # cv2.imshow("ImageFrame",imgRegion)
    # Make a copy of the points
    center_points_prev_frame = center_points_cur_frame.copy()
    print(danger_car_id)
    print(danger_car_pt)
    key = cv2.waitKey(0)
    if key == 27:
        break



cap.release()
cv2.destroyAllWindows()


Ultralytics YOLOv8.0.26  Python-3.10.9 torch-1.11.0+cpu CPU


(1920, 1080)


YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

0: 384x640 8 cars, 127.0ms
Speed: 0.0ms pre-process, 127.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


[]
{1186: 1}



0: 384x640 9 cars, 139.5ms
Speed: 2.8ms pre-process, 139.5ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 9 cars, 143.1ms
Speed: 0.0ms pre-process, 143.1ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)



...

[]
{1186: 5, 1193: 29}



0: 384x640 10 cars, 1 truck, 154.5ms
Speed: 0.0ms pre-process, 154.5ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)


[]
{1186: 5, 1193: 30}



0: 384x640 10 cars, 1 truck, 142.1ms
Speed: 11.5ms pre-process, 142.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


[1193]
{1186: 5, 1193: 31}


In [2]:
from ultralytics import YOLO
from IPython.display import display, Image
import cv2
import numpy as np
# from object_detection import ObjectDetection
import math
# from sort import *

In [1]:
from sort import *
 

In [19]:
# from ultralytics import YOLO
# from IPython.display import display, Image
# import cv2
# import numpy as np
# # from object_detection import ObjectDetection
# import math
# from sort import *

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

def dist(P, A, B):
    area = abs ( (A[0] - P[0]) * (B[1] - P[1]) - (A[1] - P[1]) * (B[0] - P[0]) )
    AB = ( (A[0] - B[0]) ** 2 + (A[1] - B[1]) ** 2 ) ** 0.5
    return ( area / AB )

f1_A=(388,821)
f1_B=(661,404) #distance 40

f2_A=(582, 876) #id 194
f2_B=(703, 415) #id 91 distance 30

f3_A=(782, 865) #id 60
f3_B=(743, 417) #id 30  distance 30

f4_A=(999, 866) #id 105
f4_B=(789, 417)  #id 85  distacne 30

f5_A=(1183, 850) #id 4
f5_B=(846, 429) #id 29 distance 30

# Initialize Object Detection
# od = ObjectDetection()
model = YOLO('yolov8n.pt')
cap = cv2.VideoCapture("2.mp4")

frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))	# 영상의 넓이(가로) 프레임
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))	# 영상의 높이(세로) 프레임
frame_size = (frameWidth, frameHeight)

print(frame_size)
# Initialize count
count = 0
center_points_prev_frame = []

danger_car_id=[]

tracking_objects = {}
track_id = 0

tracker = Sort(max_age=10, min_hits=3, iou_threshold=0.3)
mask = cv2.imread("mask.png")

while True:
    retval, frame = cap.read()
    # imgRegion = cv2.bitwise_and(frame,mask)
    count += 1
    if not(retval):
        break

    # Point current frame
    center_points_cur_frame = []

    # Detect objects on frame
    results = model(frame,stream=True)
    detections = np.empty((0,5))
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cx = int((x1+x2) / 2)
            cy = int((y1+y2) / 2)
            center_points_cur_frame.append((cx, cy, 'Danger!'))
            conf = math.ceil((box.conf[0]*100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]
            #print("FRAME N°", count, " ", x, y, w, h)
            # cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
            if currentClass == "car" or currentClass == "truck" or currentClass == "bus" \
                    or currentClass == "motorbike" or currentClass == "person" and conf > 0.3:
                cv2.rectangle(frame, (x1, y1), (x2,y2), (0, 255, 0), 2) 
                currentArray = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((detections,currentArray))

    # Only at the beginning we compare previous and current frame
    

    resultsTracker=tracker.update(detections)
    
    for result in resultsTracker:
        x1,y1,x2,y2,Id = result
        cx = int((x1+x2) / 2)
        cy = int((y1+y2) / 2)
        Id = int(Id)
        # print(cx,cy,Id)
                
    # for object_id, pt in tracking_objects.items():
        # if dist((cx,cy), f1_A, f1_B) < 30:
        #     tracking_objects[Id]=(cx,cy,'f1')
        
        # elif dist((cx,cy), f2_A, f2_B) < 30:
        #     tracking_objects[Id]=(cx,cy,'f2')
            
        # elif dist((cx,cy), f3_A, f3_B) < 30:
        #     tracking_objects[Id]=(cx,cy,'f3')
        
        # elif dist((cx,cy), f4_A, f4_B) < 30:
        #     tracking_objects[Id]=(cx,cy,'f4')
        
        # elif dist((cx,cy), f5_A, f5_B) < 30:
        #     tracking_objects[Id]=(cx,cy,'f5')
        
        # else:
        #     tracking_objects[Id]=(cx,cy,'Danger!')
        
            
            
    # for object_id, pt in tracking_objects.items():
    #     tmp= pt[2]+'['+str(object_id)+']'
    #     cv2.circle(frame, (pt[0],pt[1]), 5, (0, 0, 255), -1)
    #     cv2.putText(frame, tmp, (pt[0], pt[1] - 7), 0, 0.8, (0, 0, 255), 2)
        
    #     if (pt[2]=='Danger!'):
    #         danger_car_id.append(object_id)
            
                
    
    

    # print("Tracking objects")
    # # print(tracking_objects)


    # print("CUR FRAME LEFT PTS")
    # print(center_points_cur_frame)
    
#     cv2.circle(frame, (1,1), 5, (0, 0, 255), -1)
    cv2.imshow("Frame", frame)
    # cv2.imshow("ImageFrame",imgRegion)
    # Make a copy of the points
    center_points_prev_frame = center_points_cur_frame.copy()

    key = cv2.waitKey(0)
    if key == 27:
        break



cap.release()
cv2.destroyAllWindows()


Ultralytics YOLOv8.0.26  Python-3.10.9 torch-1.11.0+cpu CPU


(920, 488)


YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

0: 352x640 2 persons, 9 cars, 4 buss, 169.4ms
Speed: 1.6ms pre-process, 169.4ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 10 cars, 4 buss, 180.7ms
Speed: 0.0ms pre-process, 180.7ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 9 cars, 5 buss, 172.7ms
Speed: 1.0ms pre-process, 172.7ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 9 cars, 7 buss, 163.7ms
Speed: 1.0ms pre-process, 163.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 9 cars, 7 buss, 159.7ms
Speed: 1.0ms pre-process, 159.7ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 person, 10 cars, 7 buss, 1 truck, 161.7ms
Speed: 1.0ms pre-process, 161.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 person, 10 cars, 6 buss, 156.5ms
Speed: 0.9ms pre-process, 156.